In [1]:
%pylab qt
%load_ext autoreload
%autoreload 2

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [10]:
# Loop through the ggems mhd files and save them to an array
import glob
import fastcat as fc
import mpl_interactions as pli

ggems_mhd_files = glob.glob(
    '/home/jericho/1-Workspace/opengate_ggems_comparison/2-fastcat_scatter_XCAT/test/out/ggems_3e09_121kVp_*scatter.mhd'
    )

ggems_mhd_files.sort()
ggems_projections = []

for ggems_mhd_file in ggems_mhd_files:
    ggems_projection, b, c = fc.utils.read_mhd(ggems_mhd_file)
    ggems_projections.append(ggems_projection.squeeze())

In [11]:
# Plot the ggems projections
plt.figure()
controls = pli.hyperslicer(np.array(ggems_projections), cmap='jet',vmax=np.percentile(np.array(ggems_projections),99),vmin=np.percentile(np.array(ggems_projections),1))

In [12]:
def downsize_block(image,block_size):
    image_size = image.shape[0]
    block_size = int(block_size)
    downsize = image_size//block_size
    image = image[:downsize*block_size,:downsize*block_size]
    image = image.reshape(downsize,block_size,downsize,block_size).mean(axis=(1,3))
    return image

In [13]:
# Zoom the ggems projections
from scipy.ndimage import zoom

ggems_projections_zoomed = []

for ggems_projection in ggems_projections:
    ggems_projection_zoomed_downsized = downsize_block(ggems_projection, 64)
    ggems_projection_zoomed = zoom(ggems_projection_zoomed_downsized, 64, order=4, mode='nearest')
    ggems_projections_zoomed.append(ggems_projection_zoomed)

# Plot the ggems projections
plt.figure(figsize=[9,3])
plt.subplot(131)
controls = pli.hyperslicer(np.array(ggems_projections_zoomed), cmap='jet',vmax=np.percentile(np.array(ggems_projections_zoomed),99),vmin=np.percentile(np.array(ggems_projections_zoomed),1))
plt.subplot(132)
c2 = pli.hyperslicer(np.array(ggems_projections), cmap='jet',controls=controls,vmax=np.percentile(np.array(ggems_projections_zoomed),99),vmin=np.percentile(np.array(ggems_projections_zoomed),1))
plt.subplot(133)
# plot the percentage difference from the max of the non-zoomed image
perc_diff = (np.array(ggems_projections_zoomed)-np.array(ggems_projections))/np.max(ggems_projections)*100
pli.hyperslicer(perc_diff, cmap='bwr',vmax=10,vmin=-10,controls=controls)

AttributeError: 'Controls' object has no attribute 'vbox'

In [102]:
mean_projection = np.mean(np.array(ggems_projections), axis=0)
mean_projection_zoom = zoom(downsize_block(mean_projection, 64),64,order=5,mode='nearest')

pli.heatmap_slicer(np.arange(0,mean_projection.shape[0]),
                   np.arange(0,mean_projection.shape[1]),
                   (mean_projection,mean_projection_zoom),
                   cmap='jet',
                   slices='both')

(<Figure size 1800x900 with 4 Axes>,
 array([<Axes: title={'center': 'heatmap_0'}, xlabel='X', ylabel='Y'>,
        <Axes: title={'center': 'heatmap_1'}, xlabel='X'>,
        <Axes: title={'center': 'Horizontal'}>,
        <Axes: title={'center': 'Vertical'}>], dtype=object))

In [101]:
# Do the same thing but for RectBivariateSpline
from scipy.interpolate import RectBivariateSpline
import numpy as np

ggems_projections_downsized_spline = []
for ggems_projection in ggems_projections_downsized:
    # Initialize the spline
    spline = RectBivariateSpline(np.arange(ggems_projection.shape[0]),np.arange(ggems_projection.shape[1]),ggems_projection,kx=4,ky=4,s=2)
    # Evaluate the spline on a finer grid
    x_fine, y_fine = np.meshgrid(np.linspace(0, ggems_projection.shape[0], ggems_projection.shape[0]*4), np.linspace(0, ggems_projection.shape[1], ggems_projection.shape[1]*4))
    z_fine = spline.ev(x_fine, y_fine)

    ggems_projections_downsized_spline.append(z_fine)

# Plot the ggems projections
plt.figure()
controls = pli.hyperslicer(np.array(ggems_projections_downsized_spline), cmap='jet')

Init signature:
RectBivariateSpline(
    x,
    y,
    z,
    bbox=[None, None, None, None],
    kx=3,
    ky=3,
    s=0,
)
Docstring:     
Bivariate spline approximation over a rectangular mesh.

Can be used for both smoothing and interpolating data.

Parameters
----------
x,y : array_like
    1-D arrays of coordinates in strictly ascending order.
    Evaluated points outside the data range will be extrapolated.
z : array_like
    2-D array of data with shape (x.size,y.size).
bbox : array_like, optional
    Sequence of length 4 specifying the boundary of the rectangular
    approximation domain, which means the start and end spline knots of
    each dimension are set by these values. By default,
    ``bbox=[min(x), max(x), min(y), max(y)]``.
kx, ky : ints, optional
    Degrees of the bivariate spline. Default is 3.
s : float, optional
    Positive smoothing factor defined for estimation condition:
    ``sum((z[i]-f(x[i], y[i]))**2, axis=0) <= s`` where f is a spline
    function. Defa

In [81]:
phantom = np.load('/home/jericho/Software/fastcat/fastcat/data/phantoms/ct_scan_head_mandible_large.npy')

In [84]:
plt.figure()
contr = pli.hyperslicer(phantom)

In [85]:
# load a binary file to an array
import numpy as np

phantom = np.load()

In [86]:
plt.figure()
pli.hyperslicer(phantom.phantom)

AttributeError: 'Controls' object has no attribute 'vbox'